In [1]:
import pandas as pd
import json
import re
import numpy as np

In [2]:
# load the dataset
json_file = open('C:/Users/ramon/OneDrive/INSEAD/auction_proj/data.json', 'r')
#, encoding='utf-8'
data = json.load(json_file)

In [3]:
# Extract the variables of the first page
name = data[0]['name'].copy()
links = data[0]['link'].copy()
birth_year = data[0]['birth_year'].copy()
birth_place = data[0]['birth_place'].copy()
death_year = data[0]['death_year'].copy()
death_place = data[0]['death_place'].copy()

In [4]:
# Get the full links of the second pages before creating the dataframe
full_links = []
for link in links:
    full = 'http://www.ifar.org/' + link
    full_links.append(full)

# Check that the variables was created successfully
full_links[:5], len(full_links)

(['http://www.ifar.org/artist_name.php?nameid=1563&published=1&inPrep=1&author=',
  'http://www.ifar.org/artist_name.php?nameid=1396&published=1&inPrep=1&author=',
  'http://www.ifar.org/artist_name.php?nameid=1397&published=1&inPrep=1&author=',
  'http://www.ifar.org/artist_name.php?nameid=3073&published=1&inPrep=1&author=',
  'http://www.ifar.org/artist_name.php?nameid=2030&published=1&inPrep=1&author='],
 2952)

In [5]:
# Create the first dataset containing all of page 1
page_one = pd.DataFrame(list(zip(name, full_links, birth_year, birth_place, death_year, death_place)),
                          columns = ['names', 'links', 'birth_year', 'birth_place', 'death_year', 'death_place'])

In [6]:
page_one.head()

,names,links,birth_year,birth_place,death_year,death_place
0,"Aachen, Hans von",http://www.ifar.org/artist_name.php?nameid=156...,"<td class=""g_center_birth_year"">1552</td>","<td class=""g_center_birth_place"">Germany</td>","<td class=""g_center_death_year"">1615</td>","<td class=""g_center_death_place"">Czech Republi..."
1,"Aalto, Alvar",http://www.ifar.org/artist_name.php?nameid=139...,"<td class=""g_center_birth_year"">1898</td>","<td class=""g_center_birth_place"">Finland</td>","<td class=""g_center_death_year"">1976</td>","<td class=""g_center_death_place"">Finland</td>"
2,"Abbati, Giuseppe",http://www.ifar.org/artist_name.php?nameid=139...,"<td class=""g_center_birth_year"">1836</td>","<td class=""g_center_birth_place"">Italy</td>","<td class=""g_center_death_year"">1868</td>","<td class=""g_center_death_place"">Italy</td>"
3,"Abdessemed, Adel",http://www.ifar.org/artist_name.php?nameid=307...,"<td class=""g_center_birth_year"">1971</td>","<td class=""g_center_birth_place"">Algeria</td>","<td class=""g_center_death_year""></td>","<td class=""g_center_death_place""></td>"
4,"Abelenda Zapata, Manuel",http://www.ifar.org/artist_name.php?nameid=203...,"<td class=""g_center_birth_year"">1889</td>","<td class=""g_center_birth_place"">Spain</td>","<td class=""g_center_death_year"">1957</td>","<td class=""g_center_death_place"">Spain</td>"


In [7]:
# Clean all of the variables in the first page and add them again to the dataset
regex = r'^(\d{4})'
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

clean_byear = []
clean_bplace = []
clean_dyear = []
clean_dplace = []

for by in page_one['birth_year']:
    f = remove_html_tags(by)
    clean_byear.append(f)
    
for bp in page_one['birth_place']:
    q = remove_html_tags(bp)
    clean_bplace.append(q)
    
for dy in page_one['death_year']:
    e = remove_html_tags(dy)
    clean_dyear.append(e)
    
for dp in page_one['death_place']:
    w = remove_html_tags(dp)
    clean_dplace.append(w)

page_one['birth_year'] = pd.Series(clean_byear)
page_one['birth_place'] = pd.Series(clean_bplace)
page_one['death_year'] = pd.Series(clean_dyear)
page_one['death_place'] = pd.Series(clean_dplace)

In [14]:
# Make sure that the variables were cleaned correctly
page_one.head()

,names,links,birth_year,birth_place,death_year,death_place
0,"Aachen, Hans von",http://www.ifar.org/artist_name.php?nameid=156...,1552,Germany,1615,Czech Republic
1,"Aalto, Alvar",http://www.ifar.org/artist_name.php?nameid=139...,1898,Finland,1976,Finland
2,"Abbati, Giuseppe",http://www.ifar.org/artist_name.php?nameid=139...,1836,Italy,1868,Italy
3,"Abdessemed, Adel",http://www.ifar.org/artist_name.php?nameid=307...,1971,Algeria,,
4,"Abelenda Zapata, Manuel",http://www.ifar.org/artist_name.php?nameid=203...,1889,Spain,1957,Spain


In [9]:
# Make a copy of the rest of the dataset
second_part = data[1:].copy()

In [10]:
len(second_part)

7961

In [11]:
# Clean all of the variables for the second and third pages
artist_one = []
box_info = []
artist_two = []
column_names = []
content = []

for s in second_part:
    for key, val in s.items():
        if key == 'artist_one':
            artist_one.append(val)
        elif key == 'box_info':
            box_info.append(val)
        elif key == 'artist_two':
            artist_two.append(val)
        elif key == 'column_names':
            column_names.append(val)
        elif key == 'content':
            content.append(val)
            
len(content), len(name)

(5009, 2952)

In [12]:
# Clean the box
box_new = []
for i in box_info:
    tempora = []
    for s in i:
        gen = s.strip()
        tempora.append(gen)
    join_them = list(filter(None, tempora))
    box_new.append(join_them)

box_new[0], len(box_new)

(['Published Catalogues',
  'Hans von Aachen, 1552-1615.',
  'click to learn more',
  'Jacoby, Joachim W.',
  'Munich ; Berlin : Deutscher Kunstverlag, 2000.'],
 2952)

In [13]:
# Clean the new artist variable
artist_three = []
for i in artist_two:
    gen = str(i).strip('Artist: ')
    artist_three.append(gen)
    
artist_three[0], len(artist_three)

('Diziani, Gaspare', 5009)

In [15]:
# Clean the new columns
cols_news = []
for i in column_names:
    tempora = []
    for s in i:
        gen = s.strip()
        tempora.append(gen)
    cols_news.append(tempora)
    
cols_news[1], len(cols_news)

(['Item Title:',
  'Author(s):',
  'ISBN:',
  'Item Number:',
  'Imprint:',
  'Language:',
  'Size:',
  'Pages:',
  'Illustrations:',
  'Concordance:',
  'Bibliography:',
  'Index:',
  'Exhibition List:',
  'Chronology:',
  'Content Note:',
  'Individual Entries Contain:',
  'Public Note:'],
 5009)

In [25]:
new_content = []
for i in content:
    tempora = []
    for s in i:
        gen = remove_html_tags(s)
        sus = gen.strip()
        tempora.append(sus)
    join_them = list(filter(None, tempora))
    new_content.append(tempora)
    
new_content[0], len(new_content)

(['Gaspare Diziani.',
  'Zugni-Tauro, Anna Paola',
  'Venice : Alfieri, 1971.',
  'Italian',
  '28 cm',
  '363',
  'numerous ills., 126 plates (16 mounted)',
  'No',
  'Yes',
  'No',
  'No',
  'No',
  'The unillustrated catalogue raisonné (with cross references to the plates and illustrations within the text) is organized by location or collection; it gives title, medium and support, dating information, dimensions, notes, and bibliographical references. \nThe organization of the lists is topographical, and by classification: works of religious character, civil commissions, and private collectors.',
  'Bibliog.; Comments',
  "Introductory material includes a biography, a discussion of the artist's critical fortunes and other essays. The catalogue raisonné presents the works as either autograph, attributed to Diziani, by one of the artist's followers of interest to historians, and lost/unlocated. There are also appendices discusssing and cataloging  Diziani's two sons' works (classified 

In [26]:
double_list = list(zip(cols_news, new_content))
type(double_list[1])

tuple

In [27]:
v = []
for i in double_list:
    v.append(len(i[0]) == len(i[1]))

v.count(False)

347

In [28]:
momentary = []

for i in cols_news:
    for s in i:
        momentary.append(s)

x = np.array(momentary)

In [29]:
print(np.unique(x))

['Artist:' 'Author Type:' 'Author(s):' 'Author:' 'Bibliography:'
 'Catalogue:' 'Chronology:' 'Committee Member 1:' 'Committee Member 2:'
 'Committee Member 3:' 'Committee Name:' 'Concordance:'
 'Contact Information:' 'Content Note:' 'Continues Unpublished'
 'Date Begun:' 'Dissertation Author:' 'Dissertation:' 'Edition:'
 'Exhibition List:' 'Fee:' 'Format:' 'Fourth Author:'
 'Frequency of Review:' 'How Related:' 'How Revises:' 'ISBN:'
 'Illustrations:' 'Imprint:' 'Index:' 'Individual Entries Contain:'
 'Item Number:' 'Item Title:' 'Language(s):' 'Language:' 'Medium/Scope:'
 'Most Recent Edition:' 'Notes:' 'Original Publication Date:' 'Pages:'
 'Preferred Contact Method:' 'Primary Sponsor Type:' 'Primary Sponsor:'
 'Project Name:' 'Public Note:' 'Publication Date:' 'Publisher:'
 'Related Author(s):' 'Related Title:' 'Related to a Published'
 'Research by:' 'Reviewing:' 'Second Author:' 'Secondary Sponsor Type:'
 'Secondary Sponsor:' 'Size:' 'Third Author:' 'Title of Publication:'
 'Trans

In [30]:
pub_or_prep = []
for i in cols_news:
    if 'Item Title:' in i:
        pub_or_prep.append(1)
    else:
        pub_or_prep.append(2)
        
pub_or_prep[:10]

[1, 1, 1, 1, 1, 2, 2, 1, 1, 1]

In [32]:
dataset = pd.DataFrame(list(zip(artist_three, pub_or_prep, zip(cols_news, new_content))), columns = ['name', 'pub_or_prep', 'col_names_content'])
dataset.head(10)

,name,pub_or_prep,col_names_content
0,"Diziani, Gaspare",1,"([Item Title:, Author:, Imprint:, Language:, S..."
1,"Dufy, Raoul",1,"([Item Title:, Author(s):, ISBN:, Item Number:..."
2,"Guidobono, Bartolomeo",1,"([Item Title:, Author(s):, ISBN:, Imprint:, La..."
3,"Heath, Frederick",1,"([Item Title:, Author:, ISBN:, Item Number:, I..."
4,"Jones, John Llewelyn",1,"([Item Title:, Author(s):, ISBN:, Imprint:, La..."
5,"Moy, Seong",2,"([Artist:, Medium/Scope:, Title of Publication..."
6,"Schönebeck, Eugen",2,"([Artist:, Medium/Scope:, Project Name:, Autho..."
7,"Paresce, Renato",1,"([Item Title:, Author:, ISBN:, Imprint:, Langu..."
8,"Patel, Jacque",1,"([Item Title:, Author(s):, ISBN:, Imprint:, La..."
9,"Schönebeck, Eugen",1,"([Item Title:, Author:, ISBN:, Imprint:, Langu..."


In [33]:
def gen_columns(series, name, new_list):
    for i in series:
        if name in i[0]:
            new_list.append(i[1][i[0].index(name)])
        else:
            new_list.append(np.nan)

In [ ]:
•	Artist name
•	Item title
•	Has this work been published or is still in preparation (this will be on the second page, once you click on the artist’s name, but before clicking on each catalogue)
•	Author name
•	ISBN
•	Imprint
•	Language
•	Size
•	Pages
•	Illustrations
•	Concordance
•	Bibliography
•	Index
•	Exhibition List
•	Chronology
•	Individuals entries contain
•	Public note


gen_columns(series, name, new_list)

In [50]:
artist = []
item_t = []
author = []
author_s = []
isbn = []
imprint = []
language = []
size = []
pages = []
illustrations = []
concordance = []
bibliography = []
index = []
exhibition_list = []
cronology = []
ind_entr_cont = []
public_note = []



gen_columns(series = dataset.col_names_content, name = 'Artist:', new_list = artist)
gen_columns(series = dataset.col_names_content, name = 'Item Title:', new_list = item_t)
gen_columns(series = dataset.col_names_content, name = 'Author:', new_list = author)
gen_columns(series = dataset.col_names_content, name = 'Author(s):', new_list = author_s)
gen_columns(series = dataset.col_names_content, name = 'ISBN:', new_list = isbn)
gen_columns(series = dataset.col_names_content, name = 'Imprint:', new_list = imprint)
gen_columns(series = dataset.col_names_content, name = 'Language:', new_list = language)
gen_columns(series = dataset.col_names_content, name = 'Size:', new_list = size)
gen_columns(series = dataset.col_names_content, name = 'Pages:', new_list = pages)
gen_columns(series = dataset.col_names_content, name = 'Illustrations:', new_list = illustrations)
gen_columns(series = dataset.col_names_content, name = 'Concordance:', new_list = concordance)
gen_columns(series = dataset.col_names_content, name = 'Bibliography:', new_list = bibliography)
gen_columns(series = dataset.col_names_content, name = 'Index:', new_list = index)
gen_columns(series = dataset.col_names_content, name = 'Exhibition List:', new_list = exhibition_list)
gen_columns(series = dataset.col_names_content, name = 'Chronology:', new_list = cronology)
gen_columns(series = dataset.col_names_content, name = 'Individual Entries Contain:', new_list = ind_entr_cont)
gen_columns(series = dataset.col_names_content, name = 'Public Note:', new_list = public_note)

In [51]:
artist[:3], item_t[:3], author[:3], author_s[:3], isbn[:3], imprint[:3]

([nan, nan, nan],
 ['Gaspare Diziani.',
  "Raoul Dufy : catalogue raisonné de l'oeuvre peint. Supplément.",
  'Bartolomeo e Domenico Guidobono.'],
 ['Zugni-Tauro, Anna Paola', nan, nan],
 [nan,
  'Guillon-Laffaille, Fanny (bibliog.)                                                            ;&amp;nbsp\r\n                                                            Laffaille, Maurice',
  'Newcome Schleier, Mary                                                            ;&amp;nbsp\r\n                                                            Cameirana, Arrigo                                                            ;&amp;nbsp\r\n                                                            Orlando, Anna'],
 [nan, '2865740056', '8880520156'],
 ['Venice : Alfieri, 1971.',
  'Paris : Éditions Louis Carré, 1985.',
  'Turin : Artema : Compagnia di belle arti, 2002.'])

In [52]:
language[:3], size[:3], pages[:3], illustrations[:3], concordance[:3], bibliography[:3]

(['Italian', 'French', nan],
 ['28 cm', '29 cm', '30 cm'],
 ['363', '212', 'xxvii, 217'],
 ['numerous ills., 126 plates (16 mounted)',
  '243 b&amp;w, 39 color',
  'b&amp;w, some color'],
 ['No', 'No', 'No'],
 ['Yes', 'Yes', 'Yes'])

In [53]:
index[:3], exhibition_list[:3], cronology[:3], ind_entr_cont[:3]

(['No', 'Yes', 'Yes'],
 ['No', 'No', 'No'],
 ['No', 'No', 'No'],
 ['Bibliog.; Comments',
  'Exhib. Hist; Bibliog.; Comments',
  'Provenance; Bibliog.; Comments'])

In [54]:
df_final = dataset.copy()

In [55]:
df_final = df_final.drop(columns='col_names_content')
df_final.head()

,name,pub_or_prep
0,"Diziani, Gaspare",1
1,"Dufy, Raoul",1
2,"Guidobono, Bartolomeo",1
3,"Heath, Frederick",1
4,"Jones, John Llewelyn",1


In [56]:
df_final['artist'] = pd.Series(artist)
df_final['item_t'] = pd.Series(item_t)
df_final['author'] = pd.Series(author)
df_final['author_s'] = pd.Series(author_s)
df_final['isbn'] = pd.Series(isbn)
df_final['imprint'] = pd.Series(imprint)
df_final['language'] = pd.Series(language)
df_final['size'] = pd.Series(size)
df_final['pages'] = pd.Series(pages)
df_final['illustrations'] = pd.Series(illustrations)
df_final['concordance'] = pd.Series(concordance)
df_final['bibliography'] = pd.Series(bibliography)
df_final['index'] = pd.Series(index)
df_final['exhibition_list'] = pd.Series(exhibition_list)
df_final['cronology'] = pd.Series(cronology)
df_final['ind_entr_cont'] = pd.Series(ind_entr_cont)
df_final['public_note'] = pd.Series(public_note)

df_final.head()

,name,pub_or_prep,artist,item_t,author,author_s,isbn,imprint,language,size,pages,illustrations,concordance,bibliography,index,exhibition_list,cronology,ind_entr_cont,public_note
0,"Diziani, Gaspare",1,NaN,Gaspare Diziani.,"Zugni-Tauro, Anna Paola",NaN,NaN,"Venice : Alfieri, 1971.",Italian,28 cm,363,"numerous ills., 126 plates (16 mounted)",No,Yes,No,No,No,Bibliog.; Comments,"Introductory material includes a biography, a ..."
1,"Dufy, Raoul",1,NaN,Raoul Dufy : catalogue raisonné de l'oeuvre pe...,NaN,"Guillon-Laffaille, Fanny (bibliog.) ...",2865740056,"Paris : Éditions Louis Carré, 1985.",French,29 cm,212,"243 b&amp;w, 39 color",No,Yes,Yes,No,No,Exhib. Hist; Bibliog.; Comments,This is considered the fifth volume of the cat...
2,"Guidobono, Bartolomeo",1,NaN,Bartolomeo e Domenico Guidobono.,NaN,"Newcome Schleier, Mary ...",8880520156,"Turin : Artema : Compagnia di belle arti, 2002.",NaN,30 cm,"xxvii, 217","b&amp;w, some color",No,Yes,Yes,No,No,Provenance; Bibliog.; Comments,This study covers painted works found in churc...
3,"Heath, Frederick",1,NaN,"The Heath Family Engravers, 1779-1878.","Heath, John",NaN,085967908X,"Aldershot : Scolar Press, 1993.",English,26 cm,v. 1: 242; v. 2: 351,40 b&amp;w ills.,No,Yes,Yes,No,No,NaN,Each volume begins with sections on the histor...
4,"Jones, John Llewelyn",1,NaN,John Llewelyn Jones : Australia's Forgotten Pa...,NaN,"Aufy, Giles ...",0646348868,"Edgecliff, N.S.W. : Corbally Stourton Contempo...",English,30 cm,279,color,No,Yes,No,No,No,NaN,NaN


In [57]:
df_final.to_csv('C:/Users/ramon/OneDrive/INSEAD/auction_proj/full_data.csv', index=False, encoding='utf-8')

In [58]:
df = pd.read_csv('C:/Users/ramon/OneDrive/INSEAD/auction_proj/full_data.csv')

In [59]:
df.head()

,name,pub_or_prep,artist,item_t,author,author_s,isbn,imprint,language,size,pages,illustrations,concordance,bibliography,index,exhibition_list,cronology,ind_entr_cont,public_note
0,"Diziani, Gaspare",1,NaN,Gaspare Diziani.,"Zugni-Tauro, Anna Paola",NaN,NaN,"Venice : Alfieri, 1971.",Italian,28 cm,363,"numerous ills., 126 plates (16 mounted)",No,Yes,No,No,No,Bibliog.; Comments,"Introductory material includes a biography, a ..."
1,"Dufy, Raoul",1,NaN,Raoul Dufy : catalogue raisonné de l'oeuvre pe...,NaN,"Guillon-Laffaille, Fanny (bibliog.) ...",2865740056,"Paris : Éditions Louis Carré, 1985.",French,29 cm,212,"243 b&amp;w, 39 color",No,Yes,Yes,No,No,Exhib. Hist; Bibliog.; Comments,This is considered the fifth volume of the cat...
2,"Guidobono, Bartolomeo",1,NaN,Bartolomeo e Domenico Guidobono.,NaN,"Newcome Schleier, Mary ...",8880520156,"Turin : Artema : Compagnia di belle arti, 2002.",NaN,30 cm,"xxvii, 217","b&amp;w, some color",No,Yes,Yes,No,No,Provenance; Bibliog.; Comments,This study covers painted works found in churc...
3,"Heath, Frederick",1,NaN,"The Heath Family Engravers, 1779-1878.","Heath, John",NaN,085967908X,"Aldershot : Scolar Press, 1993.",English,26 cm,v. 1: 242; v. 2: 351,40 b&amp;w ills.,No,Yes,Yes,No,No,NaN,Each volume begins with sections on the histor...
4,"Jones, John Llewelyn",1,NaN,John Llewelyn Jones : Australia's Forgotten Pa...,NaN,"Aufy, Giles ...",0646348868,"Edgecliff, N.S.W. : Corbally Stourton Contempo...",English,30 cm,279,color,No,Yes,No,No,No,NaN,NaN
